In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

## **1. Data Definition**

### Customers Data

In [ ]:
demands = [0, 0, 5, 20, 10, 20, 85, 65, 30, 20, 70, 30, 0]

### Vehicles Data

In [ ]:
num_vehicles = 6
vehicle_capacity = 100

### Routes Costs

In [ ]:
# Updated distance matrix
# Routes costs (updated distance matrix)
distance_matrix = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # Distances from depot 0 to all nodes (including depot 0 and 12)
    [0, 13, 6, 55, 93, 164, 166, 168, 169, 231, 212, 0, 0],  # Distances from depot 0 to customers
    [0, 13, 0, 11, 66, 261, 175, 177, 179, 180, 239, 208, 0],  # Distances from customer 1 to other customers
    [0, 6, 11, 0, 60, 97, 168, 171, 173, 174, 239, 209, 0],  # Distances from customer 2 to other customers
    [0, 55, 66, 60, 0, 82, 113, 115, 117, 117, 295, 265, 0],  # Distances from customer 3 to other customers
    [0, 93, 261, 97, 82, 0, 113, 115, 117, 118, 333, 302, 0],  # Distances from customer 4 to other customers
    [0, 164, 175, 168, 113, 113, 0, 6, 7, 2, 403, 374, 0],  # Distances from customer 5 to other customers
    [0, 166, 177, 171, 115, 115, 6, 0, 8, 7, 406, 376, 0],  # Distances from customer 6 to other customers
    [0, 168, 179, 173, 117, 117, 7, 8, 0, 3, 408, 378, 0],  # Distances from customer 7 to other customers
    [0, 169, 180, 174, 117, 118, 2, 7, 3, 0, 409, 379, 0],  # Distances from customer 8 to other customers
    [0, 231, 239, 239, 295, 333, 403, 406, 408, 409, 0, 46, 0],  # Distances from customer 9 to other customers
    [0, 212, 208, 209, 265, 302, 374, 376, 378, 379, 46, 0, 0],  # Distances from customer 10 to other customers
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  # Distances from depot 12 to all nodes (including depot 0 and 12)
])

distances = pd.DataFrame(distance_matrix, index=range(0, 13), columns=range(0, 13))
distances

## **2. Graph**

In [ ]:
# initialize the graph
G = nx.Graph()

# Add nodes (customers and depots) to the graph
for i in range(1, 12):
    G.add_node(i)

# Add edges (connections between nodes) to the graph
for i in range(1, 12):
    for j in range(i + 1, 12):
        if distance_matrix[i, j] != np.inf:
            G.add_edge(i, j, weight=distance_matrix[i, j])

# Draw the graph
pos = nx.spring_layout(G)  # Define node positions
nx.draw(G, pos, with_labels=True, node_size=500, node_color='skyblue', font_size=10, font_weight='bold')  # Draw nodes
labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels, font_size=5)  # Draw edge labels
plt.title('Customer Graph with Distances')
plt.show()

## **3. Genetic Algorithm**

In [ ]:
# Genetic Algorithm parameters
population_size = 50
num_generations = 100
crossover_rate = 0.8
mutation_rate = 0.2

def fitness_function(routes):
    total_cost = 0
    unserved_demand = 0
    
    # Create a dictionary to keep track of the number of times each customer is visited
    visit_count = {i: 0 for i in range(1, 12)}
    
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
            
            # Update the visit count for each customer in the route
            if 0 < route[i] < 12:
                visit_count[route[i]] += 1
    
    # Calculate the unserved demand penalty
    for customer, demand in enumerate(demands[1:11], start=1):
        if visit_count[customer] == 0:
            unserved_demand += demand
    
    return total_cost + unserved_demand

def generate_initial_population():
    population = []
    for _ in range(population_size):
        routes = []
        remaining_customers = list(range(1, 12))
        for _ in range(num_vehicles):
            route = [0]
            capacity = vehicle_capacity
            while capacity >= sum([demands[i] for i in remaining_customers]):
                if not remaining_customers:
                    break
                next_customer = random.choice(remaining_customers)
                if capacity >= demands[next_customer]:
                    route.append(next_customer)
                    remaining_customers.remove(next_customer)
                    capacity -= demands[next_customer]
            route.append(12)
            routes.append(route)
        population.append(routes)
    return population

'''
def generate_initial_population():
    population = []
    for _ in range(population_size):
        routes = []
        remaining_customers = list(range(1, 12))
        for _ in range(num_vehicles):
            route = [0]
            capacity = vehicle_capacity
            while remaining_customers:
                next_customer = random.choice(remaining_customers)
                if capacity >= demands[next_customer]:
                    route.append(next_customer)
                    remaining_customers.remove(next_customer)
                    capacity -= demands[next_customer]
            route.append(12)
            routes.append(route)
        population.append(routes)
    return population
'''

def crossover(parent1, parent2):
    child1 = [route[:] for route in parent1]
    child2 = [route[:] for route in parent2]
    
    for i in range(num_vehicles):
        if random.random() < 0.5:
            child1[i], child2[i] = child2[i], child1[i]
    
    return child1, child2

def mutate(routes):
    for i in range(num_vehicles):
        if random.random() < 0.5:
            route = routes[i]
            if len(route) > 2:
                pos1, pos2 = random.sample(range(1, len(route) - 1), 2)
                route[pos1], route[pos2] = route[pos2], route[pos1]
    return routes

def genetic_algorithm():
    population = generate_initial_population()
    
    for generation in range(num_generations):
        new_population = []
        
        for _ in range(population_size // 2):
            parent1 = random.choice(population)
            parent2 = random.choice(population)
            
            if random.random() < crossover_rate:
                child1, child2 = crossover(parent1, parent2)
            else:
                child1, child2 = parent1[:], parent2[:]
            
            if random.random() < mutation_rate:
                child1 = mutate(child1)
            if random.random() < mutation_rate:
                child2 = mutate(child2)
            
            new_population.append(child1)
            new_population.append(child2)
        
        population = new_population
    
    best_solution = min(population, key=fitness_function)
    best_cost = fitness_function(best_solution)
    
    return best_solution, best_cost


## **4. Genetic Algorithm Execution**

In [ ]:
# Run the genetic algorithm
best_routes, best_cost = genetic_algorithm()

# Print the optimal routes and their cost
print("Optimal Routes:")
for route in best_routes:
    print(route)
print("Total Cost:", best_cost)

## **5. Optimal Route Visualization**

## **6. Results Analysis**